In [1]:
import pandas as pd

In [7]:
data_path = "Task_B_Dataset.csv"
df = pd.read_csv(data_path)
df.head()

,filename,para_id,sent_id,raw_text
0,cl02,28,1,"I/ppss couldn't/md* see/vb him/ppo ,/, but/cc ..."
1,cj10,9,2,Its/pp$ life/nn history/nn is/bez much/ql simp...
2,cb05,41,3,"And/cc the/at pay/nn ,/, of/in course/nn ,/, w..."
3,ck03,11,16,The/at only/ap one/cd who/wps would/md have/hv...
4,cj65,1,4,And/cc though/cs in/in his/pp$ later/jjr years...


In [3]:
import re
import json

def extract_phrases(pos_tags):
    phrases = []
    i = 0
    while i < len(pos_tags):
        if pos_tags[i] == "jj":
            j = i + 1
            while j < len(pos_tags) and pos_tags[j] == "nn":
                j += 1
                
            if j > i + 1:
                phrase = " ".join(pos_tags[i:j])
                phrases.append((i, j - 1, phrase))  
            i = j
        else:
            i += 1

    largest_phrases = []
    if phrases:
        max_end = -1
        for start, end, phrase in phrases:
            if start > max_end:
                largest_phrases.append((start, end, phrase))
                max_end = end

    return largest_phrases


def generate_output(df):
    output = {
        "pattern": "pattern 2",
        "sents": []
    }

    for _, row in df.iterrows():
        filename = row["filename"]
        para_id = row["para_id"]
        sent_id = row["sent_id"]
        raw_text = row["raw_text"]

        # Parsing sentence into (word, pos) pairs
        tokens = raw_text.split()
        words = [t.split("/")[0] for t in tokens]
        pos_tags = [t.split("/")[1] for t in tokens]

        # Extract phrases
        matches = extract_phrases(pos_tags)

        # Sentence entry
        sentence_entry = {
            "filename": filename,
            "para_id": para_id,
            "sent_id": sent_id,
            "sent_text": " ".join(words),
            "phrases": []
        }

        # Matching phrases
        for start, end, phrase in matches:
            begin_idx = len(" ".join(words[:start])) + (1 if start > 0 else 0)
            end_idx = len(" ".join(words[:end + 1]))
            sentence_entry["phrases"].append({
                "begin": begin_idx,
                "end": end_idx,
                "text": " ".join(words[start:end + 1]),
                "phrase_type": phrase
            })

        if sentence_entry["phrases"]:
            output["sents"].append(sentence_entry)

    return output

result_json = generate_output(df) 

In [4]:
with open("pattern2.json", "w") as fp:
    json.dump(result_json , fp, indent = 4) 

In [6]:
with open("pattern2.json", "r") as fp:
    pattern2_json = json.load(fp)   
pattern2_json

{'pattern': 'pattern 2',
 'sents': [{'filename': 'ck03',
   'para_id': 11,
   'sent_id': 16,
   'sent_text': 'The only one who would have him was his cripple , the strange unhappy woman who became his wife .',
   'phrases': [{'begin': 62,
     'end': 75,
     'text': 'unhappy woman',
     'phrase_type': 'jj nn'}]},
  {'filename': 'cj65',
   'para_id': 1,
   'sent_id': 4,
   'sent_text': 'And though in his later years he revised his poems many times , the revisions did not alter the essential nature of the style which he had established before he was thirty ; ;',
   'phrases': [{'begin': 96,
     'end': 112,
     'text': 'essential nature',
     'phrase_type': 'jj nn'}]},
  {'filename': 'cp06',
   'para_id': 43,
   'sent_id': 3,
   'sent_text': 'He had no great prejudice against shills ; ;',
   'phrases': [{'begin': 10,
     'end': 25,
     'text': 'great prejudice',
     'phrase_type': 'jj nn'}]},
  {'filename': 'cf21',
   'para_id': 11,
   'sent_id': 5,
   'sent_text': 'Such equipment